# Neural Network Regression 
In this notebook, we will use a neural network to predict the price of a house and see how it performs.

In [5]:
# Libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configurations
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

# Load data
df = pd.read_csv('../data/processed/rent_Valencia.csv', index_col=0)
df.head()

,price,numPhotos,floor,rooms,bathrooms,size,parkingSpacePrice,exterior,renew,new_development,hasParkingSpace,isParkingSpaceIncludedInPrice,isFinished,hasLift,hasPlan,has360,has3DTour,hasVideo,propertyType,cluster
propertyCode,,,,,,,,,,,,,,,,,,,,
97942302,850.0,23,0,5,2,140.0,0.0,False,False,False,False,False,True,False,False,False,False,False,countryHouse,3
97956954,900.0,18,1,3,3,100.0,0.0,True,False,False,True,True,True,True,False,False,False,False,duplex,0
98200619,1200.0,26,2,3,2,145.0,0.0,True,False,False,False,False,True,True,False,False,False,False,flat,0
98015046,1800.0,21,0,3,2,103.0,0.0,True,False,False,False,False,True,True,False,False,False,True,flat,0
97530419,850.0,27,5,3,1,76.0,0.0,False,False,False,False,False,True,True,False,False,False,True,flat,0


In [6]:
# Transform cluster to categorical
df['cluster'] = df['cluster'].map({0: 'central', 1: 'south', 2: 'north', 3: 'west'})

In [7]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

num_col = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_col = X_train.select_dtypes(include=['object']).columns
bool_features = X_train.select_dtypes(include=['bool']).columns

preprocessor = ColumnTransformer(
    [
    ('scaler', StandardScaler(), num_col),
    ('onehot', OneHotEncoder(), cat_col)
    ],
    remainder='passthrough'
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_col)
labels = np.concatenate([num_col, encoded_cat, bool_features])
df_train = pd.DataFrame(X_train, columns=labels, index=train_index)
df_test = pd.DataFrame(X_test, columns=labels, index=test_index)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3334 entries, 97840274 to 97951630
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   numPhotos                      3334 non-null   float64
 1   floor                          3334 non-null   float64
 2   rooms                          3334 non-null   float64
 3   bathrooms                      3334 non-null   float64
 4   size                           3334 non-null   float64
 5   parkingSpacePrice              3334 non-null   float64
 6   propertyType_chalet            3334 non-null   float64
 7   propertyType_countryHouse      3334 non-null   float64
 8   propertyType_duplex            3334 non-null   float64
 9   propertyType_flat              3334 non-null   float64
 10  propertyType_penthouse         3334 non-null   float64
 11  propertyType_studio            3334 non-null   float64
 12  cluster_central                3334 n

First we will try with Scikit-Learn librarie MLPRegressor and see the results.

In [4]:
regr = MLPRegressor(
    hidden_layer_sizes=(100, 100),
    max_iter=5000, 
    alpha=0.0001,
    solver='adam', 
    verbose=True, 
    tol=0.000000001, 
    random_state=42
    )

regr = regr.fit(X_train, y_train)

Iteration 1, loss = 891471.76530408
Iteration 2, loss = 886614.37806626
Iteration 3, loss = 875923.84116972
Iteration 4, loss = 853133.99492548
Iteration 5, loss = 810195.49955094
Iteration 6, loss = 740528.79958390
Iteration 7, loss = 640949.52576640
Iteration 8, loss = 517080.29154045
Iteration 9, loss = 386801.67001580
Iteration 10, loss = 271240.27536803
Iteration 11, loss = 196013.65856573
Iteration 12, loss = 161427.13008087
Iteration 13, loss = 151697.09055081
Iteration 14, loss = 148228.22537501
Iteration 15, loss = 145849.57980510
Iteration 16, loss = 143558.18140938
Iteration 17, loss = 141585.90862742
Iteration 18, loss = 139702.07993529
Iteration 19, loss = 137997.22458396
Iteration 20, loss = 136516.70681392
Iteration 21, loss = 135026.47336865
Iteration 22, loss = 133664.56434516
Iteration 23, loss = 132413.27126661
Iteration 24, loss = 131240.65597718
Iteration 25, loss = 130175.74405500
Iteration 26, loss = 129181.34375968
Iteration 27, loss = 128271.77842535
Iteration 

In [5]:
y_pred = regr.predict(X_test)

# Evaluate the model
print('R2 score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

R2 score: 0.3581571622675772
MSE: 224093.06892311593
MAE: 328.7986150210319
RMSE: 473.38469443267377


So for the basic model of a neural network, it seems that it not perform well... But it is pretty close to our best random forest model so with some tuning we can get a better result.

In [6]:
from msilib.schema import Directory
from unicodedata import name
import keras_tuner as kt
from keras_tuner import HyperModel

class MyModel(HyperModel):
    
    def build(self, hp): 
        model = tf.keras.Sequential()
        
        # Tuning the number of layers and the number of neurons in each layer
        layers = hp.Int('layers', min_value=1, max_value=5, step=1)
        neurons = hp.Int('neurons', min_value=32, max_value=512, step=32)

        # Tuning the optimizer and the learning rate
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
        learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

        # Tuning layer weights and biases
        weight_init = hp.Choice('weight_init', ['glorot_uniform', 'glorot_normal', 'he_uniform', 'he_normal'])
        bias_init = hp.Choice('bias_init', ['zeros', 'ones', 'random_uniform', 'random_normal'])

        # Tuning the activation function for each layer
        activation = hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])

        # Tuning the last layer activation function
        last_activation = hp.Choice('last_activation', ['linear', 'sigmoid', 'tanh'])

        # Tuning the dropout rate
        dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.05)

        for i in range(layers):
            if i == 0:
                model.add(tf.keras.layers.Dense(neurons, 
                                                kernel_initializer=weight_init, 
                                                bias_initializer=bias_init,
                                                activation=activation,
                                                input_shape=(X_train.shape[1],)))
            else:
                model.add(tf.keras.layers.Dense(neurons, 
                                                kernel_initializer=weight_init, 
                                                bias_initializer=bias_init,
                                                activation=activation))

            model.add(tf.keras.layers.Dropout(dropout))

        model.add(tf.keras.layers.Dense(1,
                                        kernel_initializer=weight_init,
                                        bias_initializer=bias_init,
                                        activation=last_activation))

        model.compile(optimizer=optimizer,
                        loss='mse',
                        metrics=['mse'])
        
        return model

    def tune(self, X_train, y_train, X_test, y_test):
        tuner = kt.Hyperband(
            self.build,
            objective='val_mse',
            max_epochs=20,
            factor=3,
            seed=42,
            hyperband_iterations=3,
            distribution_strategy=tf.distribute.MirroredStrategy(),
            project_name = 'tuning/hyperband_tuning'
        )
        
        tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
        return tuner

In [7]:
my_nn = MyModel()
tuner = my_nn.tune(X_train, y_train, X_test, y_test)

Trial 90 Complete [00h 00m 23s]
val_mse: 211290.515625

Best val_mse So Far: 211290.515625
Total elapsed time: 00h 11m 23s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.get_best_hyperparameters()[0].values

{'layers': 5,
 'neurons': 448,
 'optimizer': 'adam',
 'learning_rate': 0.001,
 'weight_init': 'he_uniform',
 'bias_init': 'random_uniform',
 'activation': 'relu',
 'last_activation': 'linear',
 'dropout': 0.15000000000000002,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [15]:
model = tuner.get_best_models()[0]
y_pred = model.predict(X_test)

print('R2 score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

27/27 [==============================] - 1s 3ms/step
R2 score: 0.39482593971767554
MSE: 211290.5285045268
MAE: 307.3441339945622
RMSE: 459.66349485740847


So this model have almost the same performance as the random forest model, let's optimize it a little more.

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import keras.backend as K 
import optuna

def objective(trial):
    K.clear_session()

    # Tuning neurons of the first hidden layer
    neuron_1 = trial.suggest_int('neuron_1', 32, 512, step=32)

    # Tuning neurons of the second hidden layer
    neuron_2 = trial.suggest_int('neuron_2', 32, 512, step=32)

    # Tuning neuron of the third hidden layer
    neuron_3 = trial.suggest_int('neuron_3', 32, 512, step=32)

    # Tuning neuron of the four hidden layer
    neuron_4 = trial.suggest_int('neuron_4', 32, 512, step=32)

    # Tuning neuron of the fifth hidden layer
    neuron_5 = trial.suggest_int('neuron_5', 32, 512, step=32)

    # Tuning learning rate
    learning_rate = float(trial.suggest_categorical('learning_rate', ['1e-2', '1e-3', '1e-4', '1e-1']))

    model = Sequential()
    
    model.add(Dense(
        neuron_1, 
        input_shape=(X_train.shape[1],), 
        activation='relu',
        kernel_initializer='he_uniform',
        bias_initializer='random_uniform'))
    model.add(Dropout(0.15000000000000002))
    model.add(Dense(
        neuron_2,
        activation='relu',
        kernel_initializer='he_uniform',
        bias_initializer='random_uniform'))
    model.add(Dropout(0.15000000000000002))
    model.add(Dense(
        neuron_3,
        activation='relu',
        kernel_initializer='he_uniform',
        bias_initializer='random_uniform'))
    model.add(Dropout(0.15000000000000002))
    model.add(Dense(
        neuron_4,
        activation='relu',
        kernel_initializer='he_uniform',
        bias_initializer='random_uniform'))
    model.add(Dropout(0.15000000000000002))
    model.add(Dense(
        neuron_5,
        activation='relu',
        kernel_initializer='he_uniform',
        bias_initializer='random_uniform'))
    model.add(Dropout(0.15000000000000002))
    model.add(Dense(1, 
    activation='linear',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
    
    model.compile(
        loss='mse', 
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['mse']
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=False)
    
    return history.history['val_loss'][-1]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2022-08-12 23:02:21,814] A new study created in memory with name: no-name-9c63330b-9cfc-4ba9-8416-feeababb6c06
[I 2022-08-12 23:02:25,889] Trial 0 finished with value: 247902.46875 and parameters: {'neuron_1': 288, 'neuron_2': 96, 'neuron_3': 96, 'neuron_4': 160, 'neuron_5': 416, 'learning_rate': '1e-2'}. Best is trial 0 with value: 247902.46875.
[I 2022-08-12 23:02:33,667] Trial 1 finished with value: 213338.046875 and parameters: {'neuron_1': 512, 'neuron_2': 256, 'neuron_3': 64, 'neuron_4': 96, 'neuron_5': 512, 'learning_rate': '1e-3'}. Best is trial 1 with value: 213338.046875.
[I 2022-08-12 23:02:37,362] Trial 2 finished with value: 1474219.25 and parameters: {'neuron_1': 128, 'neuron_2': 416, 'neuron_3': 64, 'neuron_4': 160, 'neuron_5': 192, 'learning_rate': '1e-1'}. Best is trial 1 with value: 213338.046875.
[I 2022-08-12 23:03:02,722] Trial 3 finished with value: 226334.140625 and parameters: {'neuron_1': 128, 'neuron_2': 384, 'neuron_3': 448, 'neuron_4': 96, 'neuron_5': 32,

In [28]:
best_model = study.best_trial
print(best_model.params)

{'neuron_1': 448, 'neuron_2': 512, 'neuron_3': 256, 'neuron_4': 224, 'neuron_5': 512, 'learning_rate': '1e-4'}


In [9]:
# So the best nn is:
nn_model = Sequential()
nn_model.add(Dense(
    448,
    input_shape=(X_train.shape[1],),
    activation='relu',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.add(Dropout(0.15000000000000002))
nn_model.add(Dense(
    512,
    activation='relu',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.add(Dropout(0.15000000000000002))
nn_model.add(Dense(
    256,
    activation='relu',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.add(Dropout(0.15000000000000002))
nn_model.add(Dense(
    224,
    activation='relu',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.add(Dropout(0.15000000000000002))
nn_model.add(Dense(
    512,
    activation='relu',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.add(Dropout(0.15000000000000002))
nn_model.add(Dense(
    1,
    activation='linear',
    kernel_initializer='he_uniform',
    bias_initializer='random_uniform'))
nn_model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['mse']
)

# Save the model
nn_model.save('../models/nn_model.h5')

In [10]:
np.sqrt(207811.0)

455.86291799180157

### Conclusion
* The neural network and our best random forest model have almost the same performance.